- Be sure current working directory is 'demo/' folder:

In [ ]:
import os
os.getcwd()

- Import required functions:

In [2]:
from sahi.slicing import slice_coco
from sahi.utils.file import load_json

from PIL import Image, ImageDraw
import matplotlib.pyplot as plt

import os
import glob
from PIL import Image
from concurrent.futures import ThreadPoolExecutor
import shutil
import multiprocessing
import tempfile
from tqdm import tqdm

import json
import cv2
from pathlib import Path

from multiprocessing import Pool

### Optimized version

In [3]:
images_train_source = '/home/bohbot/ultralytics/datasets/mos/all_mos_new/images/train'
images_val_source = '/home/bohbot/ultralytics/datasets/mos/all_mos_new/images/val'
labels_train_source = '/home/bohbot/ultralytics/datasets/mos/all_mos_new/labels/train'
labels_val_source = '/home/bohbot/ultralytics/datasets/mos/all_mos_new/labels/val'

In [4]:
all_image_paths = (
    [image_path for image_path in glob.iglob(f"{images_train_source}/*") 
     if image_path.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif', '.tiff'))] +
    [image_path for image_path in glob.iglob(f"{images_val_source}/*") 
     if image_path.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif', '.tiff'))]
)

all_annots_paths = (
    [annot_path for annot_path in glob.iglob(f"{labels_train_source}/*") 
     if annot_path.endswith('txt')] +
    [annot_path for annot_path in glob.iglob(f"{labels_val_source}/*") 
     if annot_path.endswith('txt')]
)

In [ ]:
annot_dict = {
    os.path.splitext(os.path.basename(annot_path))[0]: annot_path
    for annot_path in all_annots_paths if 'test' not in annot_path
}

image_annot_paths = [
    (image_path, annot_dict[os.path.splitext(os.path.basename(image_path))[0]])
    for image_path in all_image_paths
    if os.path.splitext(os.path.basename(image_path))[0] in annot_dict
]

len(image_annot_paths), image_annot_paths[:2]

In [6]:
def process_file(file_paths, large_files_dir):
    try:
        image_path, annot_path = file_paths

        if not os.path.exists(image_path) or not os.path.exists(annot_path):
            print(f"Missing file: {image_path} or {annot_path}")
            return

        with Image.open(image_path) as img:
            width, height = img.size
            if width > 1000 or height > 1000:
                shutil.copy(image_path, large_files_dir)
                shutil.copy(annot_path, large_files_dir)
            else:
                return
    except Exception as e:
        print(f"Error processing {file_paths}: {e}")


def copy_list(image_annot_paths, large_files_dir):
    os.makedirs(large_files_dir, exist_ok=True)
    copied_count = 0
    for file_paths in tqdm(image_annot_paths):
        process_file(file_paths, large_files_dir)
        copied_count += 1
    print(f"Total files copied: {copied_count}")

def copy_large_images(image_annot_paths, large_files_dir):
    copied_count = 0
    with ThreadPoolExecutor(max_workers=multiprocessing.cpu_count()-2) as executor:
        results = list(executor.map(
            lambda x: process_file(x, large_files_dir),
            image_annot_paths
        ))
    print(f"Total files copied: {copied_count}")


In [7]:
source_directory_or_paths = image_annot_paths # folder with both images and txt files
# large_files_directory = '/home/bohbot/Evyatar/git/crop_sahi/large_files' # copies only the large images to a different folder
batch_output_directory = '/home/bohbot/Evyatar/git/crop_sahi/large_images/images' # where to save the crops

In [ ]:
copy_list(image_annot_paths, batch_output_directory)

## 1. convert yolo annotations to coco

In [9]:
def process_image(args):
    image_filename, images_dir, yolo_annotations_dir = args[:3]
    category_mapping = args[3]

    image_path = os.path.join(images_dir, image_filename)
    annotation_filename = os.path.splitext(image_filename)[0] + ".txt"
    annotation_path = os.path.join(yolo_annotations_dir, annotation_filename)

    if not os.path.exists(annotation_path):
        return None, None, None

    image = cv2.imread(image_path)
    if image is None:
        return None, None, None

    height, width, _ = image.shape
    image_id = hash(image_filename)

    image_data = {
        "id": image_id,
        "file_name": image_filename,
        "height": height,
        "width": width
    }

    annotations = []
    categories = []

    with open(annotation_path, "r") as f:
        for line in f:
            line_data = line.strip().split()
            if len(line_data) < 5:
                continue

            category_id = int(line_data[0])
            x_center, y_center, bbox_width, bbox_height = map(float, line_data[1:])

            if category_id not in category_mapping:
                category_mapping[category_id] = f"category_{category_id}"
                categories.append({
                    "id": category_id,
                    "name": category_mapping[category_id],
                    "supercategory": "none"
                })

            x_min = (x_center - bbox_width / 2) * width
            y_min = (y_center - bbox_height / 2) * height
            bbox_width *= width
            bbox_height *= height

            annotations.append({
                "id": len(annotations) + 1,
                "image_id": image_id,
                "category_id": category_id,
                "bbox": [x_min, y_min, bbox_width, bbox_height],
                "area": bbox_width * bbox_height,
                "iscrowd": 0
            })

    return annotations, image_data, categories


def convert_yolo_to_coco(yolo_annotations_dir, images_dir, output_json_path):
    os.makedirs(output_json_path, exist_ok=True)
    coco_format = {"images": [], "categories": [], "annotations": []}
    category_mapping = {}

    image_filenames = [
        f for f in os.listdir(images_dir)
        if f.lower().endswith((".jpg", ".png", ".jpeg"))
    ]

    args = [
        (image_filename, images_dir, yolo_annotations_dir, category_mapping)
        for image_filename in image_filenames
    ]

    with Pool(processes=os.cpu_count()-2) as pool:
        results = list(
            tqdm(pool.imap(process_image, args), total=len(args), desc="Processing images")
        )

    for annotations, image_data, categories in results:
        if annotations and image_data:
            coco_format["annotations"].extend(annotations)
            coco_format["images"].append(image_data)
        if categories:
            coco_format["categories"].extend(categories)

    category_ids = set()
    unique_categories = []
    for category in coco_format["categories"]:
        if category["id"] not in category_ids:
            category_ids.add(category["id"])
            unique_categories.append(category)
    coco_format["categories"] = unique_categories

    with open(f"{output_json_path}/coco_annotations.json", "w") as json_file:
        json.dump(coco_format, json_file, indent=4)

In [10]:
coco_annotations_path = f"{batch_output_directory.replace('/images','/coco')}"
sliced_images_path = f"{batch_output_directory.replace('/images','/sliced')}"
temp_image_dir = f"{batch_output_directory.replace('/images','/temp_processing')}"

In [ ]:
convert_yolo_to_coco(batch_output_directory, batch_output_directory, coco_annotations_path)

## 2. Slicing COCO Dataset into Grids

- To slice a COCO dataset annoations an images, we need to specify slice parameters. In this example we will ice images into 256x256 grids overlap ratio of 0.2:

In [ ]:
n_files = len(glob.glob(f"{batch_output_directory}/*txt"))
n_files

In [13]:
batch = n_files
json_out =f"{coco_annotations_path}/coco_annotations.json"
sliced_dir = sliced_images_path

In [ ]:
from dask import bag as db

def process_file(image_file):
    try:
        slice_coco(
            coco_annotation_file_path=json_out,
            image_dir=batch_output_directory,
            output_coco_annotation_file_name="sliced_annotations",
            ignore_negative_samples=False,
            output_dir=sliced_dir,
            slice_height=640,
            slice_width=640,
            overlap_height_ratio=0.2,
            overlap_width_ratio=0.2,
            min_area_ratio=0.1,
            verbose=False
        )
    except Exception:
        return image_file
    return None

image_files = [f for f in glob.iglob(f"{batch_output_directory}/*") if 'txt' not in f]

missing_files = db.from_sequence(image_files, npartitions=8).map(process_file).compute()

missing_files = [file for file in missing_files if file]

if missing_files:
    with open("missing_files.log", "w") as log_file:
        log_file.write("\n".join(missing_files))


 71%|███████   | 6915/9779 [08:30<03:06, 15.36it/s]

- Convert back to yolo

In [ ]:
from sahi.utils.coco import Coco

for i in sorted(os.listdir(batch_output_directory)):
  batch = os.path.join(batch_output_directory,i)
    
  sliced_dir = os.path.join(batch,"sliced/")
  print(sliced_dir)
  json_sliced = os.path.join(sliced_dir,"sliced_coco.json")
  # init Coco object
  coco = Coco.from_coco_dict_or_path(json_sliced, image_dir=sliced_dir)

  # export converted YoloV5 formatted dataset into given output_dir with a 85% train/15% val split
  coco.export_as_yolov5(
    output_dir=os.path.join(source_directory,"sliced_images_with_yolo_format"),
    train_split_rate=1,
      disable_symlink=True
  )


- Visualize sliced annotations on sliced images:

In [ ]:
f, axarr = plt.subplots(4, 5, figsize=(13,13))
img_ind = 0
for row_ind in range(4):
    for column_ind in range(5):
        # read image
        img = Image.open("demo_data/sliced/" + coco_dict["images"][img_ind]["file_name"]).convert('RGBA')
        # iterate over all annotations
        for ann_ind in range(len(coco_dict["annotations"])):
            # find annotations that belong the selected image
            if coco_dict["annotations"][ann_ind]["image_id"] == coco_dict["images"][img_ind]["id"]:
                # convert coco bbox to pil bbox
                xywh = coco_dict["annotations"][ann_ind]["bbox"]
                xyxy = [xywh[0], xywh[1], xywh[0]+xywh[2], xywh[1]+xywh[3]]
                # visualize bbox over image
                ImageDraw.Draw(img, 'RGBA').rectangle(xyxy, width=5)
            axarr[row_ind, column_ind].imshow(img)
        img_ind += 1
